# mini_proj_01
## 20181531 김연주
### 최종: 94.09

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

In [3]:
from sklearn.utils import shuffle

X_train = shuffle(X_train_full, random_state= 42)
y_train = shuffle(y_train_full, random_state= 42)

In [4]:
X_train = X_train_full / 255.
y_train = y_train_full.astype(float)

In [5]:
X_train

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [6]:
y_train

array([9., 0., 0., ..., 3., 0., 5.])

In [7]:
print(X_train.shape, y_train.shape)

(60000, 28, 28) (60000,)


In [8]:
print(X_test.shape)

(10000, 28, 28)


In [9]:
def build_model():
    from functools import partial

    DefaultConv2D = partial(keras.layers.Conv2D,
                            kernel_size=2, kernel_initializer = 'lecun_normal',
                            activation='selu', padding="SAME")

    model = keras.models.Sequential([
        DefaultConv2D(filters=64, kernel_size=3, input_shape=[28, 28,1]),
        # keras.layers.MaxPooling2D(pool_size=2),
        keras.layers.BatchNormalization(),
        DefaultConv2D(filters=64),
        keras.layers.BatchNormalization(),
        DefaultConv2D(filters=128),
        keras.layers.BatchNormalization(),
        DefaultConv2D(filters=128),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D(pool_size=2),
#         DefaultConv2D(filters=128),
#         keras.layers.BatchNormalization(),
#         keras.layers.MaxPooling2D(pool_size=2),
        DefaultConv2D(filters=256),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D(pool_size=2),
        DefaultConv2D(filters=256),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D(pool_size=2),
        
        keras.layers.Flatten(),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(units=128, activation='selu'),
        keras.layers.BatchNormalization(),
#         keras.layers.Dropout(0.2),
        keras.layers.Dense(units=64, activation='selu'),
        keras.layers.BatchNormalization(),
        #.layers.Dropout(0.3),
        keras.layers.Dense(units=10, activation='softmax'),
    ])
    
    
    model.compile(loss="sparse_categorical_crossentropy",
                  optimizer="adam",
                  metrics=["accuracy"])
    return model

In [10]:
k = 10
num_val = len(X_train) // k
num_epochs = 100
all_scores = []


for i in range(k):
    print('processing fold #', i)

    X_val = X_train[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate(
        [X_train[:i * num_val],
         X_train[(i + 1) * num_val:]],
        axis=0)
    y_train_part = np.concatenate(
        [y_train[:i * num_val],
         y_train[(i + 1) * num_val:]],
        axis=0)

    model = build_model()
    with tf.device("/device:GPU:0"):
        model.fit(X_train_part, y_train_part,
                  epochs=num_epochs, verbose=1)
#     history = model.fit(X_train_part, y_train_part,
#                   epochs=num_epochs, verbose=1)
    val_loss, val_acc = model.evaluate(X_val, y_val)
    all_scores.append(val_acc)

processing fold # 0
Epoch 1/100
1688/1688 [==============================] - 54s 30ms/step - loss: 0.3649 - accuracy: 0.8710
Epoch 2/100
1688/1688 [==============================] - 51s 30ms/step - loss: 0.2473 - accuracy: 0.9114
Epoch 3/100
1688/1688 [==============================] - 51s 30ms/step - loss: 0.2094 - accuracy: 0.9240
Epoch 4/100
1688/1688 [==============================] - 51s 30ms/step - loss: 0.1797 - accuracy: 0.9355
Epoch 5/100
1688/1688 [==============================] - 51s 30ms/step - loss: 0.1539 - accuracy: 0.9445
Epoch 6/100
1688/1688 [==============================] - 51s 31ms/step - loss: 0.1331 - accuracy: 0.9521
Epoch 7/100
1688/1688 [==============================] - 52s 31ms/step - loss: 0.1127 - accuracy: 0.9594
Epoch 8/100
1688/1688 [==============================] - 52s 31ms/step - loss: 0.0981 - accuracy: 0.9636
Epoch 9/100
1688/1688 [==============================] - 52s 31ms/step - loss: 0.0800 - accuracy: 0.9714
Epoch 10/100
1688/1688 [===========

1688/1688 [==============================] - 53s 31ms/step - loss: 0.0109 - accuracy: 0.9962
Epoch 55/100
1688/1688 [==============================] - 53s 31ms/step - loss: 0.0093 - accuracy: 0.9966
Epoch 56/100
1688/1688 [==============================] - 53s 31ms/step - loss: 0.0103 - accuracy: 0.9966
Epoch 57/100
1688/1688 [==============================] - 53s 31ms/step - loss: 0.0104 - accuracy: 0.9963
Epoch 58/100
1688/1688 [==============================] - 53s 31ms/step - loss: 0.0099 - accuracy: 0.9964
Epoch 59/100
1688/1688 [==============================] - 52s 31ms/step - loss: 0.0098 - accuracy: 0.9966
Epoch 60/100
1688/1688 [==============================] - 53s 31ms/step - loss: 0.0090 - accuracy: 0.9970
Epoch 61/100
1688/1688 [==============================] - 53s 31ms/step - loss: 0.0121 - accuracy: 0.9960
Epoch 62/100
1688/1688 [==============================] - 53s 31ms/step - loss: 0.0082 - accuracy: 0.9973
Epoch 63/100
1688/1688 [==============================] - 5

1688/1688 [==============================] - 53s 31ms/step - loss: 0.1357 - accuracy: 0.9509
Epoch 7/100
1688/1688 [==============================] - 53s 31ms/step - loss: 0.1151 - accuracy: 0.9580
Epoch 8/100
1688/1688 [==============================] - 53s 31ms/step - loss: 0.0972 - accuracy: 0.9650
Epoch 9/100
1688/1688 [==============================] - 53s 31ms/step - loss: 0.0837 - accuracy: 0.9696
Epoch 10/100
1688/1688 [==============================] - 53s 31ms/step - loss: 0.0705 - accuracy: 0.9739
Epoch 11/100
1688/1688 [==============================] - 53s 31ms/step - loss: 0.0627 - accuracy: 0.9773
Epoch 12/100
1688/1688 [==============================] - 53s 31ms/step - loss: 0.0556 - accuracy: 0.9797
Epoch 13/100
1688/1688 [==============================] - 53s 31ms/step - loss: 0.0501 - accuracy: 0.9817
Epoch 14/100
1688/1688 [==============================] - 52s 31ms/step - loss: 0.0476 - accuracy: 0.9831
Epoch 15/100
1688/1688 [==============================] - 53s 

1688/1688 [==============================] - 53s 31ms/step - loss: 0.0101 - accuracy: 0.9966
Epoch 60/100
1688/1688 [==============================] - 53s 31ms/step - loss: 0.0102 - accuracy: 0.9964
Epoch 61/100
1688/1688 [==============================] - 53s 31ms/step - loss: 0.0095 - accuracy: 0.9967
Epoch 62/100
1688/1688 [==============================] - 53s 31ms/step - loss: 0.0089 - accuracy: 0.9971
Epoch 63/100
1688/1688 [==============================] - 53s 31ms/step - loss: 0.0087 - accuracy: 0.9968
Epoch 64/100
1688/1688 [==============================] - 53s 31ms/step - loss: 0.0094 - accuracy: 0.9969
Epoch 65/100
1688/1688 [==============================] - 53s 31ms/step - loss: 0.0092 - accuracy: 0.9969
Epoch 66/100
1688/1688 [==============================] - 53s 31ms/step - loss: 0.0080 - accuracy: 0.9971
Epoch 67/100
1688/1688 [==============================] - 53s 31ms/step - loss: 0.0097 - accuracy: 0.9967
Epoch 68/100
1688/1688 [==============================] - 5

1688/1688 [==============================] - 52s 31ms/step - loss: 0.0614 - accuracy: 0.9779
Epoch 12/100
1688/1688 [==============================] - 52s 31ms/step - loss: 0.0535 - accuracy: 0.9805
Epoch 13/100
1688/1688 [==============================] - 52s 31ms/step - loss: 0.0488 - accuracy: 0.9817
Epoch 14/100
1688/1688 [==============================] - 52s 31ms/step - loss: 0.0460 - accuracy: 0.9836
Epoch 15/100
1688/1688 [==============================] - 52s 31ms/step - loss: 0.0415 - accuracy: 0.9851
Epoch 16/100
1688/1688 [==============================] - 52s 31ms/step - loss: 0.0346 - accuracy: 0.9872
Epoch 17/100
1688/1688 [==============================] - 52s 31ms/step - loss: 0.0382 - accuracy: 0.9861
Epoch 18/100
1688/1688 [==============================] - 52s 31ms/step - loss: 0.0314 - accuracy: 0.9884
Epoch 19/100
1688/1688 [==============================] - 52s 31ms/step - loss: 0.0332 - accuracy: 0.9879
Epoch 20/100
1688/1688 [==============================] - 5

1688/1688 [==============================] - 53s 31ms/step - loss: 0.0090 - accuracy: 0.9965
Epoch 65/100
1688/1688 [==============================] - 53s 31ms/step - loss: 0.0092 - accuracy: 0.9969
Epoch 66/100
1688/1688 [==============================] - 53s 31ms/step - loss: 0.0081 - accuracy: 0.9969
Epoch 67/100
1688/1688 [==============================] - 53s 31ms/step - loss: 0.0088 - accuracy: 0.9969
Epoch 68/100
1688/1688 [==============================] - 53s 31ms/step - loss: 0.0092 - accuracy: 0.9970
Epoch 69/100
1688/1688 [==============================] - 53s 31ms/step - loss: 0.0083 - accuracy: 0.9971
Epoch 70/100
1688/1688 [==============================] - 53s 31ms/step - loss: 0.0069 - accuracy: 0.9974
Epoch 71/100
1688/1688 [==============================] - 53s 32ms/step - loss: 0.0087 - accuracy: 0.9971
Epoch 72/100
1688/1688 [==============================] - 53s 31ms/step - loss: 0.0082 - accuracy: 0.9973
Epoch 73/100
1688/1688 [==============================] - 5

1688/1688 [==============================] - 53s 31ms/step - loss: 0.0344 - accuracy: 0.9882
Epoch 17/100
1688/1688 [==============================] - 53s 31ms/step - loss: 0.0361 - accuracy: 0.9876
Epoch 18/100
1688/1688 [==============================] - 53s 31ms/step - loss: 0.0356 - accuracy: 0.9876
Epoch 19/100
1688/1688 [==============================] - 53s 32ms/step - loss: 0.0319 - accuracy: 0.9879
Epoch 20/100
1688/1688 [==============================] - 53s 32ms/step - loss: 0.0299 - accuracy: 0.9893
Epoch 21/100
1688/1688 [==============================] - 53s 32ms/step - loss: 0.0295 - accuracy: 0.9899
Epoch 22/100
1688/1688 [==============================] - 53s 32ms/step - loss: 0.0253 - accuracy: 0.9910
Epoch 23/100
1688/1688 [==============================] - 53s 32ms/step - loss: 0.0269 - accuracy: 0.9908
Epoch 24/100
1688/1688 [==============================] - 53s 32ms/step - loss: 0.0243 - accuracy: 0.9914
Epoch 25/100
1688/1688 [==============================] - 5

In [11]:
all_scores

[0.9394999742507935,
 0.9383333325386047,
 0.9398333430290222,
 0.9403333067893982,
 0.9366666674613953,
 0.9413333535194397,
 0.9461666941642761,
 0.9409999847412109,
 0.9403333067893982,
 0.9456666707992554]

In [12]:
np.mean(all_scores)

0.9409166634082794